In [ ]:
import os  # 파일 확인용 모듈 추가

import time
from datetime import datetime, timedelta
import random
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ==========================================
# 1. 강력한 속도 최적화 설정
# ==========================================
chrome_options = Options()

# [최적화 1] 헤드리스 모드 (화면 안 띄움 -> 리소스 절약)
# 혹시 에러 나서 화면을 보고 싶으면 이 줄을 주석 처리(#) 하세요.
chrome_options.add_argument("--headless=new") 

# [최적화 2] 이미지 로딩 차단 (속도 대폭 향상)
chrome_options.add_argument("--blink-settings=imagesEnabled=false")

# 기타 설정
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# [최적화 3] 페이지 로딩 전략 'eager' (글자만 뜨면 바로 진행)
chrome_options.page_load_strategy = 'eager'

driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10) # 스마트 대기 설정 (최대 10초)

# 경로 설정: 프로젝트 루트 / data / data/임시
project_root = os.path.dirname(os.getcwd())  # .../DSP/project
data_dir = os.path.join(project_root, "data")
temp_dir = os.path.join(data_dir, "임시")
os.makedirs(temp_dir, exist_ok=True)

# 설정값
MAX_PAGE_LIMIT = 150
SAVE_INTERVAL = 50 # 50개 수집할 때마다 중간 저장

base_url = "https://web.joongna.com/search/%EC%95%84%EC%9D%B4%ED%8F%B0%2015?keywordSource=INPUT_KEYWORD&minPrice=160000&maxPrice=4000000&sort=RECENT_SORT&saleYn=SALE_Y"

product_urls = []
page = 1

print("=== [1단계] 고속 링크 수집 시작 (Headless & Eager Mode) ===")
print("화면이 뜨지 않아도 백그라운드에서 실행 중입니다. 잠시만 기다려주세요...")

# ==========================================
# 2. 링크 수집
# ==========================================
while True:
    try:
        target_url = f"{base_url}&page={page}"
        driver.get(target_url)
        
        # [스마트 대기] 상품 리스트가 뜰 때까지만 기다림
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/product/']")))
        except:
            pass

        # 스크롤 (빠르게)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5) 
        
        all_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='/product/']")
        
        # 페이지네이션 위치 찾기 (없으면 pass)
        nav_location_y = 9999999
        try:
            pagination_nav = driver.find_element(By.TAG_NAME, "nav")
            nav_location_y = pagination_nav.location['y']
        except: pass

        # 필터링 및 저장
        valid_links = []
        for link in all_links:
            try:
                if link.location['y'] > 0 and link.location['y'] < nav_location_y:
                    valid_links.append(link.get_attribute("href"))
            except: pass
        
        if not valid_links: # 필터링 실패 시 전체 수집
             valid_links = [l.get_attribute("href") for l in all_links]

        # 중복 제거 후 추가
        added_count = 0
        for href in valid_links:
            if href and href not in product_urls:
                product_urls.append(href)
                added_count += 1

        print(f"[Page {page}] {added_count}개 추가 (누적: {len(product_urls)}개)")

        if added_count == 0:
            print("더 이상 새로운 상품 없음. 종료.")
            break

        page += 1
        if page > MAX_PAGE_LIMIT:
            print("최대 페이지 도달.")
            break

    except Exception as e:
        print(f"[Page {page}] 치명적 에러: {e}")
        break

print(f"\n=== 링크 수집 완료: 총 {len(product_urls)}개 ===")

# ==========================================
# 3. 상세 데이터 고속 수집 (이어하기 기능 추가)
# ==========================================
print("\n=== [2단계] 상세 정보 고속 크롤링 시작 (이어하기 모드) ===")

# [NEW] 이미 수집된 URL 목록 만들기
collected_urls = set()

# data/임시 폴더 안의 temp 파일들만 대상으로 함
temp_files = []
if os.path.isdir(temp_dir):
    temp_files = [
        os.path.join(temp_dir, f)
        for f in os.listdir(temp_dir)
        if f.startswith('iphone15_temp_resume_') and f.endswith('.csv')
    ]

if temp_files:
    print(f"👉 기존 임시 파일 {len(temp_files)}개를 발견했습니다. 중복 수집을 방지합니다.")
    for t_file in temp_files:
        try:
            # CSV를 읽어서 URL 컬럼만 가져옴
            already_df = pd.read_csv(t_file)
            collected_urls.update(already_df['url'].tolist())
        except:
            pass
    print(f"👉 이미 수집된 {len(collected_urls)}개의 상품은 건너뜁니다.")

data_list = []
start_count = len(collected_urls) # 누적 카운트용

for idx, url in enumerate(product_urls):
    # [NEW] 이미 수집한 URL이면 스킵
    if url in collected_urls:
        continue

    try:
        driver.get(url)
        
        # [최적화] 제목이 뜰 때까지만 기다림
        try:
            wait.until(EC.presence_of_element_located((By.TAG_NAME, 'h1')))
        except:
            pass 
        
        driver.implicitly_wait(0.5) 

        # --- (데이터 추출 로직 기존과 동일) ---
        try: title = driver.find_element(By.TAG_NAME, 'h1').text
        except: title = ""

        try:
            price_raw = driver.find_element(By.XPATH, "//div[contains(@class, 'font-bold') and contains(text(), '원')]").text
            price = int(price_raw.replace("원", "").replace(",", "").strip())
        except: price = 0

        # === 날짜만 추출 & 정규화 ===
        # 1) 화면에 실제로 보이는 게시물 정보 영역(span)에서 우선 추출
        date_raw = ""
        try:
            info_span = driver.find_element(
                By.XPATH,
                "//span[contains(@class, 'text-jnGray-500') and contains(text(), '조회')]"
            )
            info_text = info_span.text.strip()
            # 예: "2025-10-26 · 조회 48 · 채팅 1 · 찜 1" 또는 "39분 전 · 조회 48 · ..."
            first_part = info_text.split('·')[0].strip()

            # YYYY-MM-DD 또는 YYYY.MM.DD 형식이면 YYYY-MM-DD로 통일
            m = re.search(r"(\d{4})[.\-]\s?(\d{1,2})[.\-]\s?(\d{1,2})", first_part)
            if m:
                y, mo, d = m.groups()
                date_raw = f"{y}-{int(mo):02d}-{int(d):02d}"
            else:
                # 상대시간("39분 전", "3시간 전", "2일 전" 등)은 그대로 임시 저장
                if re.search(r"\d{1,2}(?:분|시간|일)\s*전", first_part):
                    date_raw = first_part
        except:
            pass

        # 2) <time datetime="..."> 태그에서 추출 (fallback)
        if not date_raw:
            try:
                t = driver.find_element(By.XPATH, "//time[@datetime]")
                date_raw = (t.get_attribute("datetime") or t.text or "").strip()
            except:
                pass

        # 3) <meta ... time> 태그에서 추출 (추가 fallback)
        if not date_raw:
            for prop in ["article:published_time","article:modified_time","og:updated_time","og:article:published_time"]:
                try:
                    m = driver.find_element(By.XPATH, f"//meta[@property='{prop}' or @name='{prop}']")
                    content = (m.get_attribute("content") or "").strip()
                    if content:
                        date_raw = content
                        break
                except:
                    pass

        # 4) 마지막 안전장치: 페이지 전체 텍스트에서 YYYY-MM-DD / YYYY.MM.DD 한 번 더 시도
        if not date_raw:
            try:
                body_text = driver.find_element(By.TAG_NAME, "body").text.replace("\n", " ")
                m = re.search(r"(\d{4})[.\-]\s?(\d{1,2})[.\-]\s?(\d{1,2})", body_text)
                if m:
                    y, mo, d = m.groups()
                    date_raw = f"{y}-{int(mo):02d}-{int(d):02d}"
            except:
                pass

        # 5) 상대시간(~분 전, ~시간 전, ~일 전)을 오늘 기준 날짜(YYYY-MM-DD)로 변환
        normalized_date = ""
        if date_raw:
            dr = date_raw.strip()

            # 이미 YYYY-MM-DD 형식이면 그대로 사용
            m = re.match(r"^(\d{4})-(\d{2})-(\d{2})$", dr)
            if m:
                normalized_date = dr
            else:
                # YYYY.MM.DD 형식이면 변환
                m = re.match(r"^(\d{4})[.](\d{1,2})[.](\d{1,2})$", dr)
                if m:
                    y, mo, d = m.groups()
                    normalized_date = f"{y}-{int(mo):02d}-{int(d):02d}"
                else:
                    # X분 전, X시간 전 -> 오늘 날짜
                    m = re.match(r"^(\d{1,2})분\s*전$", dr)
                    if m:
                        base = datetime.now().date()
                        normalized_date = base.strftime("%Y-%m-%d")
                    else:
                        m = re.match(r"^(\d{1,2})시간\s*전$", dr)
                        if m:
                            base = datetime.now().date()
                            normalized_date = base.strftime("%Y-%m-%d")
                        else:
                            # X일 전 -> 오늘 - X일
                            m = re.match(r"^(\d{1,2})일\s*전$", dr)
                            if m:
                                days = int(m.group(1))
                                base = datetime.now().date() - timedelta(days=days)
                                normalized_date = base.strftime("%Y-%m-%d")

        # 최종적으로 CSV에 저장할 값은 normalized_date가 있으면 그 값을 사용
        if normalized_date:
            date_raw = normalized_date

        try: description = driver.find_element(By.TAG_NAME, "article").text
        except: description = ""
            
        listing_id = url.split("/")[-1]

        # 상태 확인
        status = "판매중"
        try:
            body_text = driver.find_element(By.TAG_NAME, "body").text
            if "판매완료" in body_text: status = "판매완료"
            elif "예약중" in body_text: status = "예약중"
        except: pass

        # 스펙 정보 추출
        spec_model = ""
        spec_color = ""
        spec_storage = ""
        try:
            try:
                model_elem = driver.find_element(By.XPATH, "//*[contains(text(), '기종')]")
                spec_model = model_elem.text.replace("기종", "").strip()
                if not spec_model: spec_model = model_elem.find_element(By.XPATH, "./..").text.replace("기종", "").strip()
            except: pass

            try:
                color_elem = driver.find_element(By.XPATH, "//*[contains(text(), '색상')]")
                spec_color = color_elem.text.replace("색상", "").strip()
                if not spec_color: spec_color = color_elem.find_element(By.XPATH, "./..").text.replace("색상", "").strip()
            except: pass

            try:
                storage_elem = driver.find_element(By.XPATH, "//*[contains(text(), '용량')]")
                spec_storage = storage_elem.text.replace("용량", "").strip()
                if not spec_storage: spec_storage = storage_elem.find_element(By.XPATH, "./..").text.replace("용량", "").strip()
            except: pass
        except: pass

        # 데이터 추가
        row_data = {
            "listing_id": listing_id,
            "status": status,
            "spec_model": spec_model,
            "spec_color": spec_color,
            "spec_storage": spec_storage,
            "price": price,
            "date_raw": date_raw,
            "title": title,
            "description": description,
            "url": url
        }
        data_list.append(row_data)
        
        # 전체 진행률 계산 (기존 수집 + 현재 수집)
        current_total = start_count + len(data_list)
        
        # 로그 출력
        if len(data_list) % 10 == 0:
            print(f"[{current_total}/{len(product_urls)}] 완료 ({status})")

        # 중간 저장 (중요: 파일명을 겹치지 않게 unique하게 생성)
        if len(data_list) % SAVE_INTERVAL == 0:
            # 기존 파일과 겹치지 않게 'resume' 붙여서 저장 (data/임시 폴더)
            temp_filename = f"iphone15_temp_resume_{current_total}.csv"
            temp_path = os.path.join(temp_dir, temp_filename)
            pd.DataFrame(data_list).to_csv(temp_path, index=False, encoding="utf-8-sig")
            print(f"--- 중간 저장 완료 ({os.path.relpath(temp_path, project_root)}) ---")
            
            # 메모리 관리를 위해 리스트 비우기 (선택사항이나 추천)
            # data_list = [] 
            # (※ 주의: data_list를 비우면 나중에 최종 저장할 때 비어있게 되므로, 
            #  이 방식은 계속 파일을 쪼개서 저장하고 나중에 합치는 방식에 적합함.
            #  일단은 안전하게 리스트 유지하되, 브라우저만 가끔 껐다 켜주면 좋음)

    except Exception as e:
        # 치명적 에러 시 중단
        if "invalid session id" in str(e) or "chrome not reachable" in str(e):
             print("!!! 브라우저 다운됨. 현재까지 데이터 저장 후 종료 !!!")
             break
        print(f"Skipping: {e}")
        continue

driver.quit()

# 남은 데이터 최종 저장
if data_list:
    df = pd.DataFrame(data_list)
    # 파일명에 timestamp 등을 넣어 겹치지 않게 함
    filename = f"iphone15_resume_final_{int(time.time())}.csv"
    final_path = os.path.join(data_dir, filename)
    cols = ["listing_id", "status", "spec_model", "spec_color", "spec_storage", "price", "date_raw", "title", "description", "url"]
    df = df[cols]
    df.to_csv(final_path, index=False, encoding="utf-8-sig")
    print(f"\n[성공] 이어하기 완료! 파일명: {os.path.relpath(final_path, project_root)}")
else:
    print("\n추가로 수집된 데이터가 없습니다.")

=== [1단계] 고속 링크 수집 시작 (Headless & Eager Mode) ===
화면이 뜨지 않아도 백그라운드에서 실행 중입니다. 잠시만 기다려주세요...
[Page 1] 81개 추가 (누적: 81개)
[Page 2] 80개 추가 (누적: 161개)
[Page 3] 80개 추가 (누적: 241개)
[Page 4] 80개 추가 (누적: 321개)
[Page 5] 80개 추가 (누적: 401개)
[Page 6] 80개 추가 (누적: 481개)
[Page 7] 80개 추가 (누적: 561개)
[Page 8] 80개 추가 (누적: 641개)
[Page 9] 80개 추가 (누적: 721개)
[Page 10] 80개 추가 (누적: 801개)
[Page 11] 80개 추가 (누적: 881개)
[Page 12] 80개 추가 (누적: 961개)
[Page 13] 80개 추가 (누적: 1041개)
[Page 14] 80개 추가 (누적: 1121개)
[Page 15] 80개 추가 (누적: 1201개)
[Page 16] 80개 추가 (누적: 1281개)
[Page 17] 80개 추가 (누적: 1361개)
[Page 18] 80개 추가 (누적: 1441개)
[Page 19] 80개 추가 (누적: 1521개)
[Page 20] 80개 추가 (누적: 1601개)
[Page 21] 80개 추가 (누적: 1681개)
[Page 22] 80개 추가 (누적: 1761개)
[Page 23] 80개 추가 (누적: 1841개)
[Page 24] 80개 추가 (누적: 1921개)
[Page 25] 80개 추가 (누적: 2001개)
[Page 26] 80개 추가 (누적: 2081개)
[Page 27] 80개 추가 (누적: 2161개)
[Page 28] 80개 추가 (누적: 2241개)
[Page 29] 80개 추가 (누적: 2321개)
[Page 30] 80개 추가 (누적: 2401개)
[Page 31] 80개 추가 (누적: 2481개)
[Page 32] 80개 추가 (누적: 2561개)
[P

In [3]:
import os
import glob
import pandas as pd

# === 임시/최종 CSV 모두 합쳐서 하나로 병합하는 전용 셀 ===
# 기준 폴더: 이 노트북 기준 ../data (프로젝트의 data 폴더)

# 1) 경로 설정
project_root = os.path.dirname(os.getcwd())  # .../DSP/project
data_dir = os.path.join(project_root, "data")
temp_dir = os.path.join(data_dir, "임시")

print("project_root:", project_root)
print("data_dir:", data_dir)
print("temp_dir:", temp_dir)

# 2) 파일 찾기
#   - 임시 파일: data/임시/iphone15_temp_resume_*.csv
#   - 최종 파일: data/iphone15_resume_final_*.csv

temp_files = []
if os.path.isdir(temp_dir):
    temp_files = sorted(glob.glob(os.path.join(temp_dir, "iphone15_temp_resume_*.csv")))

final_files = sorted(glob.glob(os.path.join(data_dir, "iphone15_resume_final_*.csv")))

print("임시 파일 개수:", len(temp_files))
print("최종 파일 개수:", len(final_files))
print()

csv_files = temp_files + final_files

if not csv_files:
    print("병합할 CSV 파일이 없습니다. 경로와 파일명을 다시 확인해 주세요.")
else:
    dfs = []
    for path in csv_files:
        try:
            df_part = pd.read_csv(path)
            print(f"{os.path.relpath(path, project_root)}: {len(df_part)}행 로드")
            dfs.append(df_part)
        except Exception as e:
            print(f"{path} 읽기 실패: {e}")

    if not dfs:
        print("어떤 CSV도 정상적으로 로드되지 않았습니다.")
    else:
        df_all = pd.concat(dfs, ignore_index=True)

        # listing_id 기준 중복 제거
        if "listing_id" in df_all.columns:
            before = len(df_all)
            df_all = df_all.drop_duplicates(subset="listing_id", keep="last")
            after = len(df_all)
            print(f"\n중복 제거 전: {before}행, 후: {after}행 (중복 {before - after}행 제거)")
        else:
            print("\n경고: 'listing_id' 컬럼이 없어 중복 제거를 건너뜁니다.")

        # 컬럼 순서를 crawling 결과와 최대한 맞추기
        cols = [
            "listing_id", "status", "spec_model", "spec_color", "spec_storage",
            "price", "date_raw", "title", "description", "url"
        ]
        existing_cols = [c for c in cols if c in df_all.columns]
        df_all = df_all[existing_cols + [c for c in df_all.columns if c not in existing_cols]]

        # 3) 최종 병합 파일 저장 (data 폴더 아래)
        out_path = os.path.join(data_dir, "iphone15_resume_final_merged.csv")
        df_all.to_csv(out_path, index=False, encoding="utf-8-sig")
        print(f"\n[완료] 통합 결과 저장: {os.path.relpath(out_path, project_root)}")
        print("최종 행 수:", len(df_all))



project_root: /Users/miin/univ/4/2/DSP/project
data_dir: /Users/miin/univ/4/2/DSP/project/data
temp_dir: /Users/miin/univ/4/2/DSP/project/data/임시
임시 파일 개수: 166
최종 파일 개수: 1

data/임시/iphone15_temp_resume_100.csv: 100행 로드
data/임시/iphone15_temp_resume_1000.csv: 1000행 로드
data/임시/iphone15_temp_resume_1050.csv: 1050행 로드
data/임시/iphone15_temp_resume_1100.csv: 1100행 로드
data/임시/iphone15_temp_resume_1150.csv: 1150행 로드
data/임시/iphone15_temp_resume_1200.csv: 1200행 로드
data/임시/iphone15_temp_resume_1250.csv: 1250행 로드
data/임시/iphone15_temp_resume_1300.csv: 1300행 로드
data/임시/iphone15_temp_resume_1350.csv: 1350행 로드
data/임시/iphone15_temp_resume_1400.csv: 1400행 로드
data/임시/iphone15_temp_resume_1450.csv: 1450행 로드
data/임시/iphone15_temp_resume_150.csv: 150행 로드
data/임시/iphone15_temp_resume_1500.csv: 1500행 로드
data/임시/iphone15_temp_resume_1550.csv: 1550행 로드
data/임시/iphone15_temp_resume_1600.csv: 1600행 로드
data/임시/iphone15_temp_resume_1650.csv: 1650행 로드
data/임시/iphone15_temp_resume_1700.csv: 1700행 로드
data/임시/iphone1

In [2]:
import os
import pandas as pd

# 프로젝트 루트 및 데이터 경로 설정
project_root = os.path.dirname(os.getcwd())  # .../DSP/project
data_dir = os.path.join(project_root, "data")

csv_path = os.path.join(data_dir, "iphone15_resume_final_merged.csv")

if not os.path.exists(csv_path):
    print(f"CSV 파일을 찾을 수 없습니다: {csv_path}")
else:
    df_merged = pd.read_csv(csv_path)

    if "date_raw" not in df_merged.columns:
        print("'date_raw' 컬럼이 없습니다.")
    else:
        # 날짜 컬럼을 datetime으로 변환
        df_merged["date_raw"] = pd.to_datetime(df_merged["date_raw"], errors="coerce")

        min_date = df_merged["date_raw"].min()
        max_date = df_merged["date_raw"].max()

        if pd.isna(min_date) or pd.isna(max_date):
            print("유효한 날짜 데이터가 없습니다.")
        else:
            print("iphone15_resume_final_merged.csv 기준")
            print("가장 오래된 날짜:", min_date.strftime("%Y-%m-%d"))
            print("가장 최근 날짜:", max_date.strftime("%Y-%m-%d"))


iphone15_resume_final_merged.csv 기준
가장 오래된 날짜: 2024-12-02
가장 최근 날짜: 2025-12-02
